<a href="https://colab.research.google.com/github/theadarshkr/Feynn-Labs-Assignment/blob/main/AdarshKumar_T_2_(CropDiseasePred%26FertRecomm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
#Installing the Kaggle library
!pip install kaggle

#Importing the Kaggle API key file
from google.colab import files
files.upload()  #Uploading our Kaggle API key file i.e. kaggle.json

#Moving the Kaggle API key file to the appropriate location
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

#Setting permissions for the Kaggle API key file
!chmod 600 ~/.kaggle/kaggle.json

#Downloading the dataset using the Kaggle API
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

#Unzipping the downloaded file
!unzip new-plant-diseases-dataset.zip

#Defining directories
train_dir = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
test_dir = "/content/test/test"

#Loading training data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

#Loading validation data
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

#Loading test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

#Retrieving class labels
class_labels = list(train_generator.class_indices.keys())

In [ ]:
#Defining the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(class_labels), activation="softmax"))

#Compiling the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

#Defining callbacks
es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
mc = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True)
tb = TensorBoard(log_dir="logs")

#Training the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=valid_generator,
    callbacks=[es, mc, tb]
)

#Evaluating the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Epoch 1/50
2197/2197 [==============================] - ETA: 0s - loss: 1.5641 - accuracy: 0.5442

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2197/2197 [==============================] - 9106s 4s/step - loss: 1.5641 - accuracy: 0.5442 - val_loss: 0.5936 - val_accuracy: 0.8120
Epoch 2/50
 163/2197 [=>............................] - ETA: 2:08:50 - loss: 0.7475 - accuracy: 0.7665

In [ ]:
#Extracting features from the last convolutional layer of the CNN model
feature_model = Model(inputs=model.input, outputs=model.layers[-4].output)

#Extracting features from training and validation sets
X_train_features = feature_model.predict(train_generator)
X_val_features = feature_model.predict(valid_generator)

#Reshaping features to a 2D array
X_train_features = np.reshape(X_train_features, (len(X_train_features), -1))
X_val_features = np.reshape(X_val_features, (len(X_val_features), -1))

#Creating and train the SVM model
svm = SVC(kernel="rbf", C=1.0, gamma="scale")
svm.fit(X_train_features, train_generator.classes)

#Evaluating the SVM model on the validation set
y_val_pred = svm.predict(X_val_features)
print("Accuracy:", accuracy_score(valid_generator.classes, y_val_pred))
print("Confusion matrix:\n", confusion_matrix(valid_generator.classes, y_val_pred))
print("Classification report:\n", classification_report(valid_generator.classes, y_val_pred))

**Fertilizer Recommendation**

In [ ]:
#Defining the dictionary mapping diseases to fertilizers and dosages
fertilizer_dict = {
 "Apple___Apple_scab": ("Urea", "0.5 kg per tree"),
 "Apple___Black_rot": ("Bordeaux mixture", "1.5 kg per 100 liters of water"),
 "Apple___Cedar_apple_rust": ("Mancozeb", "2.5 kg per hectare"),
 "Apple___healthy": ("None", "None"),
 "Blueberry___healthy": ("None", "None"),
 "Cherry_(including_sour)___healthy": ("None", "None"),
 "Cherry_(including_sour)___Powdery_mildew": ("Sulfur", "3 kg per hectare"),
 "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot": ("Azoxystrobin", "0.5 liter per hectare"),
 "Corn_(maize)___Common_rust_": ("Propiconazole", "1 liter per hectare"),
 "Corn_(maize)___healthy": ("None", "None"),
 "Corn_(maize)___Northern_Leaf_Blight": ("Pyraclostrobin", "0.75 liter per hectare"),
 "Grape___Black_rot": ("Copper hydroxide", "2 kg per hectare"),
 "Grape___Esca_(Black_Measles)": ("Potassium bicarbonate", "5 kg per hectare"),
 "Grape___healthy": ("None", "None"),
 "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": ("Chlorothalonil", "2.5 kg per hectare")
}
#Defining the function to recommend fertilizer based on the predicted disease label
def recommend_fertilizer(label):
    if label in fertilizer_dict:
        return fertilizer_dict[label]
    elif label == "Healthy":
        return "None", "None"
    else:
        return "Unknown", "Unknown"

#Getting predictions on the test data
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

#Recommending fertilizer for each predicted label
for i, label_idx in enumerate(predicted_labels):
    label = class_labels[label_idx]
    fertilizer, dosage = recommend_fertilizer(label)
    print(f"For test image {test_generator.filenames[i]}, the predicted disease is {label}.")
    print(f"The recommended fertilizer is {fertilizer} and the dosage is {dosage}.")